# PyCity Schools Analysis
1.Looking at the tuition more families are going for District than Charter.
2.If we look at the data,(passing both the math and the reading tests)it os significant that HALF of the students are not doing good in the larger schools.


In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data["student_name"] = student_data["student_name"].str.replace(word,"")

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
#Calculate all values to be displayed in Df

Total_Schools =len(school_data["school_name"])
student_count = len(student_data["Student ID"])
# Calculate the total budget
total_budget = school_data["budget"].sum()
# Calculate the average (mean) math score
average_math_score = student_data["math_score"].mean()
# Calculate the average (mean) reading score
average_reading_score = student_data["reading_score"].mean()
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete.loc[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_reading_count=school_data_complete.loc[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100

In [3]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools":[Total_Schools],
                                 "Total Students":[student_count],
                                 "Total Budget":[total_budget],
                                 "Avg Math Score":[average_math_score],
                                 "Passing Math %":[passing_math_percentage],
                                 "Avg Reading Score":[average_reading_score],
                                 "Passing Reading %":[passing_reading_percentage],
                                 "Overall Passing rate":[overall_passing_rate]})
                                                             
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.0f}".format)
district_summary["Avg Math Score"] = district_summary["Avg Math Score"].map("${:,.2f}".format)
district_summary["Passing Math %"] = district_summary["Passing Math %"].map("${:,.2f}%".format)
district_summary["Passing Reading %"] = district_summary["Passing Reading %"].map("${:,.2f}%".format)
district_summary["Avg Reading Score"] = district_summary["Avg Reading Score"].map("${:,.2f}%".format)
district_summary["Overall Passing rate"] = district_summary["Overall Passing rate"].map("${:,.2f}%".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Avg Math Score,Passing Math %,Avg Reading Score,Passing Reading %,Overall Passing rate
0,15,"39,170","$24,649,428",$78.99,$74.98%,$81.88%,$85.81%,$65.17%


## School Summary

In [4]:
student_data["reading pass"]=student_data["reading_score"]>70
student_data["math pass"]=student_data["math_score"]>70
student_data["Passed Math & Reading"]=((student_data["reading pass"])&(student_data["math pass"]))

student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,reading pass,math pass,Passed Math & Reading
0,0,Paul Bradley,M,9th,Huang High School,66,79,False,True,False
1,1,Victor Smith,M,12th,Huang High School,94,61,True,False,False
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,True,False,False
3,3,Richard Scott,M,12th,Huang High School,67,58,False,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True,True


In [5]:
#Now creating a Summary Table for all of the Schools
school_summary=school_data
school_summary=school_summary.rename(columns={"type":"School Type",
                                             "size":"Total Students",
                                             "budget":"Total School Budget"})
school_summary["Per Student Budget"]=school_summary["Total School Budget"]/school_summary["Total Students"]
school_summary.drop("School ID",axis=1,inplace=True)

first_gp=student_data.groupby(["school_name"])

In [6]:
#calculate the Math and Reading Passing for the summary table

school_math=first_gp['math_score'].mean()
school_reading=first_gp['reading_score'].mean()
math_num=first_gp['math pass'].sum()
reading_num=first_gp['reading pass'].sum()
both_num=first_gp['Passed Math & Reading'].sum()
school_math_df=pd.DataFrame(school_math)
school_math_df["math pass num"]= math_num
school_math_df["Passed Both"]=both_num
school_math_df.reset_index(inplace=True)
school_reading_df=pd.DataFrame(school_reading)
school_reading_df["read pass num"]=reading_num
school_reading_df.reset_index(inplace=True)
math_and_read=pd.merge(school_math_df,school_reading_df,)
school_summary=pd.merge(school_summary,math_and_read,on="school_name")
school_summary["% Passing Math"]=school_summary["math pass num"]/school_summary["Total Students"]*100
school_summary["% Passing Reading"]=school_summary["read pass num"]/school_summary["Total Students"]*100
school_summary["Overall Passing Rate"]=((school_summary['% Passing Math']+school_summary['% Passing Reading'])/2)
school_summary["% Passed Both"]=school_summary['Passed Both']/school_summary["Total Students"]*100

In [7]:
#Additional Formatting for the table
final_school_summary=school_summary
final_school_summary=final_school_summary.rename(columns={"school":"School",
                                                         "math_score":"Avg. Math Score",
                                                         "reading_score":"Avg. Reading Score"})
final_school_summary.drop("math pass num",axis=1,inplace=True)
final_school_summary.drop("read pass num",axis=1,inplace=True)
final_school_summary=final_school_summary.round(2)
final_school_summary["Total Students"]=final_school_summary["Total Students"].map("{0:,d}".format)
final_school_summary["% Passing Math"]=final_school_summary["% Passing Math"].map("{0:,.2f}%".format)
final_school_summary["% Passing Reading"]=final_school_summary["% Passing Reading"].map("{0:,.2f}%".format)
final_school_summary["Overall Passing Rate"]=final_school_summary["Overall Passing Rate"].map("{0:,.2f}%".format)
final_school_summary["Total School Budget"]=final_school_summary["Total School Budget"].map("${0:,.0f}".format)
final_school_summary["Per Student Budget"]=final_school_summary["Per Student Budget"].map("${0:,.0f}".format)
final_school_summary["% Passed Both"]=final_school_summary["% Passed Both"].map("{0:,.2f}%".format)
#Summary for all schools
final_school_summary=final_school_summary.sort_values("school_name")
final_school_summary.reset_index(inplace=True)
final_school_summary

,index,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Passed Both,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
0,7,Bailey High School,District,"4,976","$3,124,928",$628,77.05,2545,81.03,64.63%,79.30%,71.97%,51.15%
1,6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,1561,83.98,89.56%,93.86%,91.71%,84.02%
2,1,Figueroa High School,District,"2,949","$1,884,411",$639,76.71,1472,81.16,63.75%,78.43%,71.09%,49.92%
3,13,Ford High School,District,"2,739","$1,763,916",$644,77.10,1405,80.75,65.75%,77.51%,71.63%,51.30%
4,4,Griffin High School,Charter,"1,468","$917,500",$625,83.35,1228,83.82,89.71%,93.39%,91.55%,83.65%
5,3,Hernandez High School,District,"4,635","$3,022,020",$652,77.29,2325,80.93,64.75%,78.19%,71.47%,50.16%
6,8,Holden High School,Charter,427,"$248,087",$581,83.80,359,83.81,90.63%,92.74%,91.69%,84.07%
7,0,Huang High School,District,"2,917","$1,910,635",$655,76.63,1456,81.18,63.32%,78.81%,71.07%,49.91%
8,12,Johnson High School,District,"4,761","$3,094,650",$650,77.07,2371,80.97,63.85%,78.28%,71.07%,49.80%
9,9,Pena High School,Charter,962,"$585,858",$609,83.84,816,84.04,91.68%,92.20%,91.94%,84.82%


## Highest-Performing Schools (by % Overall Passing)

In [8]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = final_school_summary.sort_values(by='Overall Passing Rate', ascending=False)
top_schools.head(5)

,index,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Passed Both,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
13,5,Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,1938,83.99,90.93%,93.25%,92.09%,84.89%
9,9,Pena High School,Charter,962,"$585,858",$609,83.84,816,84.04,91.68%,92.20%,91.94%,84.82%
14,10,Wright High School,Charter,"1,800","$1,049,400",$583,83.68,1520,83.96,90.28%,93.44%,91.86%,84.44%
1,6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,1561,83.98,89.56%,93.86%,91.71%,84.02%
6,8,Holden High School,Charter,427,"$248,087",$581,83.80,359,83.81,90.63%,92.74%,91.69%,84.07%


## Bottom Performing Schools (By % Overall Passing)

In [9]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = final_school_summary.sort_values(by='Overall Passing Rate', ascending=True)
bottom_schools.head(5)

,index,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Passed Both,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
10,11,Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,1977,80.74,64.07%,77.74%,70.91%,49.44%
7,0,Huang High School,District,"2,917","$1,910,635",$655,76.63,1456,81.18,63.32%,78.81%,71.07%,49.91%
8,12,Johnson High School,District,"4,761","$3,094,650",$650,77.07,2371,80.97,63.85%,78.28%,71.07%,49.80%
2,1,Figueroa High School,District,"2,949","$1,884,411",$639,76.71,1472,81.16,63.75%,78.43%,71.09%,49.92%
5,3,Hernandez High School,District,"4,635","$3,022,020",$652,77.29,2325,80.93,64.75%,78.19%,71.47%,50.16%


## Math Scores by Grade

In [34]:
by_grade=student_data
by_grade['grade']=by_grade['grade'].astype('category', ['9th','10th','11th','12th'])
math_scores_by_grade= round(by_grade.pivot_table(index="school_name",columns="grade",values="math_score"),2)
math_scores_by_grade

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,77.00,77.52,76.49,77.08
Cabrera High School,83.15,82.77,83.28,83.09
Figueroa High School,76.54,76.88,77.15,76.40
Ford High School,77.67,76.92,76.18,77.36
Griffin High School,84.23,83.84,83.36,82.04
Hernandez High School,77.34,77.14,77.19,77.44
Holden High School,83.43,85.00,82.86,83.79
Huang High School,75.91,76.45,77.23,77.03
Johnson High School,76.69,77.49,76.86,77.19


## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = 
tenth_grader_reading_scores = 
eleventh_grader_reading_scores = 
twelfth_grader_reading_scores = 

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = 

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

In [36]:
#Table to look at Reading scores across all schools by grade
reading_scores_grade= round(by_grade.pivot_table(index="school_name",columns="grade",values="reading_score"),2)
reading_scores_grade

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.91,80.95,80.91,81.30
Cabrera High School,84.25,83.79,84.29,83.68
Figueroa High School,81.41,80.64,81.38,81.20
Ford High School,81.26,80.40,80.66,80.63
Griffin High School,83.71,84.29,84.01,83.37
Hernandez High School,80.66,81.40,80.86,80.87
Holden High School,83.32,83.82,84.70,83.68
Huang High School,81.51,81.42,80.31,81.29
Johnson High School,80.77,80.62,81.23,81.26


## Scores by School Spending

In [40]:
# Establish the bins 

spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
binned_spend=school_summary
binned_spend["Spending Ranges per Student"]=pd.cut(binned_spend["Per Student Budget"],spending_bins,labels)
spend_grouped_schools=pd.pivot_table(binned_spend, index=["Spending Ranges per Student"])
per_student_spend=round(spend_grouped_schools[['math_score','reading_score','% Passing Math','% Passing Reading','Overall Passing Rate','% Passed Both']],2)
per_student_spend=per_student_spend.rename(columns={"math_score":"Avg. Math Score","reading_score":"Avg. Reading Score"})

per_student_spend

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary